# Imputation der Geo-Daten

## 1. Datenvorbereitung und Bereinigung

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from pyproj import Transformer
from geopy.geocoders import Nominatim
import time

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.max_info_columns', 10000)   # zeigt alle Spaltennamen in info()
pd.set_option('display.max_info_rows', 200000)     # zeigt Zeileninfo, wenn nötig


# Pfad zur Datei 
path = Path("dataset/311_Service_Requests_2024.csv")
df = pd.read_csv(path)

## 2. Einteilung der Spalten mit Geodaten und Missing Values

In [ ]:
# Nur Geodaten-Spalten analysieren
geo_cols = ['STREET_ADDRESS','STREET_NUMBER', 'STREET_NAME', 'STREET_DIRECTION', 'STREET_TYPE','CITY',
            'STATE', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'X_COORDINATE', 'Y_COORDINATE', 'LOCATION',]

missing_count = df[geo_cols].isnull().sum()
missing_percent = df[geo_cols].isnull().mean() * 100

# Erstellung DataFrames für Übersicht
missing_df = pd.DataFrame({
    'Anzahl fehlender Werte': missing_count,
    'Prozent fehlender Werte': missing_percent
})

# Sortierung nach Anzahl fehlender Werte (absteigend)
missing_df = missing_df[missing_df['Anzahl fehlender Werte'] > 0]
missing_df = missing_df.sort_values(by='Anzahl fehlender Werte', ascending=False)

print("Fehlende Werte in Geodaten-Spalten:")
print(missing_df)

Fehlende Werte in Geodaten-Spalten:
                  Anzahl fehlender Werte  Prozent fehlender Werte
ZIP_CODE                          210954                11.022039
CITY                              152869                 7.987182
STATE                             152869                 7.987182
STREET_TYPE                        17446                 0.911528
COMMUNITY_AREA                      1809                 0.094518
WARD                                1741                 0.090965
POLICE_DISTRICT                     1677                 0.087621
LATITUDE                            1062                 0.055488
LONGITUDE                           1062                 0.055488
LOCATION                            1062                 0.055488
Y_COORDINATE                        1015                 0.053032
X_COORDINATE                        1015                 0.053032
STREET_DIRECTION                     934                 0.048800
STREET_NUMBER                        835

Wir sehen, dass ca. 1000 Geo-Daten fehlen, also wo die SR aufgenommen wurden. Wir versuchen jetzt aus den gegebenen Spalten (z B STREET_ADDRESS) andere Spalten (z B LATITUDE) herauszubekommen. 

## 3. Funktion für die Imputation der Koordinaten

In [ ]:
def geocode_missing_addresses(df):
    geolocator = Nominatim(user_agent="chicago_311_geocoder_12345")
    
    missing = df['LATITUDE'].isna() & df['STREET_ADDRESS'].notna()
    to_geocode = df[missing].copy()
    
    total = len(to_geocode)
    print(f"Geocodiere {total} Adressen (ca. {total // 60 + 1} Minuten)\n")
    
    results = []
    success = 0
    
    for i, (idx, row) in enumerate(to_geocode.iterrows(), 1):
        try:
            address = f"{row['STREET_ADDRESS']}, Chicago, IL, USA"
            location = geolocator.geocode(address, timeout=10)
            
            if location:
                results.append({
                    'index': idx,
                    'latitude': location.latitude,
                    'longitude': location.longitude
                })
                success += 1
                
        except Exception:
            pass
        
        time.sleep(1)
        
        if i % 50 == 0 or i == total:
            print(f"{i}/{total} ({i/total*100:.1f}%) - Gefunden: {success}")

    # Koordinaten in DF schreiben
    for result in results:
        df.loc[result['index'], 'LATITUDE'] = result['latitude']
        df.loc[result['index'], 'LONGITUDE'] = result['longitude']
        df.loc[result['index'], 'LOCATION'] = f"({result['latitude']}, {result['longitude']})"
    
    print(f"\nErfolgreich: {success}/{total} ({success/total*100:.1f}%)")
    
    # Rückgabe zusätzlich: Liste der neuen Indizes
    new_indices = [r['index'] for r in results]
    return df, new_indices


## 4. Funktion zum Hinzufügen der gefundenen Missing Values und zum Berechnen der Lat/Long aus X/Y und umgekehrt

In [1]:
def impute_coordinates(df):
    print("\nCODE 2: Koordinaten-Imputation gestartet\n")
    
    missing_address = df['STREET_ADDRESS'].isna()
    has_components = df['STREET_NUMBER'].notna() & df['STREET_NAME'].notna()
    to_reconstruct = missing_address & has_components
    
    if to_reconstruct.sum() > 0:
        print(f"Rekonstruiere {to_reconstruct.sum()} Adressen...")
        for idx in df[to_reconstruct].index:
            parts = []
            if pd.notna(df.loc[idx, 'STREET_NUMBER']):
                parts.append(str(int(df.loc[idx, 'STREET_NUMBER'])))
            if pd.notna(df.loc[idx, 'STREET_DIRECTION']):
                parts.append(df.loc[idx, 'STREET_DIRECTION'])
            if pd.notna(df.loc[idx, 'STREET_NAME']):
                parts.append(df.loc[idx, 'STREET_NAME'])
            if pd.notna(df.loc[idx, 'STREET_TYPE']):
                parts.append(df.loc[idx, 'STREET_TYPE'])
            
            if parts:
                df.loc[idx, 'STREET_ADDRESS'] = ' '.join(parts)
    
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3435", always_xy=True)
    missing_xy = df['X_COORDINATE'].isna() & df['LATITUDE'].notna()

    if missing_xy.sum() > 0:
        for idx in df[missing_xy].index:
            lat = df.loc[idx, 'LATITUDE']
            lon = df.loc[idx, 'LONGITUDE']
            if pd.notna(lat) and pd.notna(lon):
                x, y = transformer.transform(lon, lat)
                df.loc[idx, 'X_COORDINATE'] = x
                df.loc[idx, 'Y_COORDINATE'] = y
    
    transformer_reverse = Transformer.from_crs("EPSG:3435", "EPSG:4326", always_xy=True)
    missing_latlon = df['LATITUDE'].isna() & df['X_COORDINATE'].notna()

    if missing_latlon.sum() > 0:
        for idx in df[missing_latlon].index:
            x = df.loc[idx, 'X_COORDINATE']
            y = df.loc[idx, 'Y_COORDINATE']
            if pd.notna(x) and pd.notna(y):
                lon, lat = transformer_reverse.transform(x, y)
                df.loc[idx, 'LATITUDE'] = lat
                df.loc[idx, 'LONGITUDE'] = lon
    
    missing_location = df['LOCATION'].isna() & df['LATITUDE'].notna()
    if missing_location.sum() > 0:
        df.loc[missing_location, 'LOCATION'] = df[missing_location].apply(
            lambda row: f"({row['LATITUDE']}, {row['LONGITUDE']})", axis=1
        )

    has_geo = df['STREET_ADDRESS'].notna() | df['LATITUDE'].notna() | df['WARD'].notna()
    missing_city = df['CITY'].isna() & has_geo
    missing_state = df['STATE'].isna() & has_geo

    df.loc[missing_city, 'CITY'] = 'Chicago'
    df.loc[missing_state, 'STATE'] = 'IL'
    
    print("\nImputation abgeschlossen")
    print(f"City gesetzt: {missing_city.sum()}, State gesetzt: {missing_state.sum()}")
    
    imputed_rows = (to_reconstruct | missing_xy | missing_latlon | missing_location | missing_city | missing_state)
    
    # Rückgabe der betroffenen Zeilen
    return df, df.loc[imputed_rows].index.tolist()


## 5. Ausführung der Funktionen und Erstellung neuer CSV files

In [ ]:
# Sicherstellen, dass das Original erhalten bleibt 
df_imputed = df.copy()

# Definierte Spalten für den finalen Export
FINAL_GEO_COLS = [
    'SR_NUMBER', 'STREET_ADDRESS', 'LATITUDE', 'LONGITUDE', 
    'X_COORDINATE', 'Y_COORDINATE', 'LOCATION', 'CITY', 'STATE',
    'STREET_NUMBER', 'STREET_NAME', 'STREET_DIRECTION', 'STREET_TYPE'
]

# Filtere, um nur die wirklich vorhandenen Spalten zu verwenden
available_final_cols = [c for c in FINAL_GEO_COLS if c in df_imputed.columns]

# 1. Imputation
df_imputed, imputed_idx = impute_coordinates(df_imputed)

# 2. Geocoding (Extern) 
df_imputed, geocoded_idx = geocode_missing_addresses(df_imputed)

# 3. Erneute Imputation nach neuen Lat/Lon 
df_imputed, imputed_idx2 = impute_coordinates(df_imputed)

# Liste aller Indizes, die in irgendeinem Schritt bearbeitet wurden
all_new_indices = sorted(set(imputed_idx) | set(geocoded_idx) | set(imputed_idx2))

print("\n--- Exporting CSVs ---")

# CSV NUR mit neu imputierten Zeilen und ALLEN Geo-Spalten
# Hier verwenden wir die gefilterte Liste available_final_cols
df_new_data = df_imputed.loc[all_new_indices, available_final_cols]
df_new_data.to_csv("dataset/imputed_geo_data_only.csv", index=False, encoding="utf-8-sig")
print("'imputed_geo_data_only.csv': Enthält NUR die Zeilen, die neu imputiert wurden.")


# (2) CSV mit ALLEN Zeilen und ALLEN Geo-Spalten (neu und alt)
df_all_data_geo_reduced = df_imputed[available_final_cols]
df_all_data_geo_reduced.to_csv("dataset/all_data_with_imputed_geo.csv", index=False, encoding="utf-8-sig")
print("all_data_with_imputed_geo.csv': Enthält ALLE Zeilen mit den finalen Geo-Daten.")


CODE 2: Koordinaten-Imputation gestartet


Imputation abgeschlossen
City gesetzt: 152238, State gesetzt: 152238
Geocodiere 221 Adressen (ca. 4 Minuten)

50/221 (22.6%) - Gefunden: 48
100/221 (45.2%) - Gefunden: 96
150/221 (67.9%) - Gefunden: 146
200/221 (90.5%) - Gefunden: 196
221/221 (100.0%) - Gefunden: 211

Erfolgreich: 211/221 (95.5%)

CODE 2: Koordinaten-Imputation gestartet


Imputation abgeschlossen
City gesetzt: 0, State gesetzt: 0

--- Exporting CSVs ---
'imputed_geo_data_only.csv': Enthält NUR die Zeilen, die neu imputiert wurden.
all_data_with_imputed_geo.csv': Enthält ALLE Zeilen mit den finalen Geo-Daten.


## 6. Erstellung einer CSV nur mit den neu gefundenen Koordinaten-Daten

In [ ]:
# 1. Filtern des DataFrames auf alle Zeilen, die an das externe Geocoding gesendet wurden (221 Adressen).
# Die Variable 'geocoded_idx' wurde von 'geocode_missing_addresses' zurückgegeben.
df_geocoding_attempts = df_imputed.loc[geocoded_idx].copy()

# 2. Definiere, was Erfolg bedeutet: LATITUDE existiert und ist NICHT NaN.
# Dies isoliert die Zeilen von den 221, bei denen der Prozess erfolgreich war.
successfully_found_coords = df_geocoding_attempts['LATITUDE'].notna()

# 3. Das finale DataFrame mit den 211 erfolgreich geocodierten Zeilen.
df_211_coords = df_geocoding_attempts.loc[successfully_found_coords, available_final_cols]

# 4. Export der CSV
output_filename = "dataset/only_211_new_coordinates.csv"
df_211_coords.to_csv(output_filename, index=False, encoding="utf-8-sig")

# 5. Ausgabe zur Bestätigung
print("\n--- Exporting only the 211 New Coordinates ---")
print(f"'{output_filename}': Enthält {len(df_211_coords)} Zeilen, die erfolgreich neue Geo-Koordinaten erhalten haben (Erwartet: 211).")

# Optional: Kurze Vorschau der ersten Zeilen
print("\n--- Vorschau (Geo-Koordinaten) ---")
print(df_211_coords[['SR_NUMBER', 'STREET_ADDRESS', 'LATITUDE', 'LONGITUDE']].head())


--- Exporting ONLY the 211 New Coordinates ---
'dataset/only_211_new_coordinates.csv': Enthält 211 Zeilen, die erfolgreich neue Geo-Koordinaten erhalten haben (Erwartet: 211).

--- Vorschau (Geo-Koordinaten) ---
             SR_NUMBER       STREET_ADDRESS   LATITUDE  LONGITUDE
358437   SR24-01894008       2159 W 92ND ST  41.726430 -87.675479
1557117  SR24-00439859  2111 W Lexington ST  41.871914 -87.679850
1557118  SR24-00439858  2111 W Lexington ST  41.871914 -87.679850
1557267  SR24-00439695     10510 W ZEMKE RD  41.994980 -87.887581
1557268  SR24-00439694     10510 W ZEMKE RD  41.994980 -87.887581


## 7. Untersuchung der Missing Values nach der Imputierung

In [ ]:
df_after = pd.read_csv("dataset/all_data_with_imputed_geo.csv") 

check_cols = [
    'STREET_ADDRESS', 'LATITUDE', 'LONGITUDE', 
    'X_COORDINATE', 'Y_COORDINATE', 'LOCATION', 'CITY', 'STATE'
]

# Filtern der Spalten, die wirklich im DataFrame sind, um Fehler zu vermeiden
available_check_cols = [c for c in check_cols if c in df_after.columns]

missing_count = df_after[available_check_cols].isnull().sum()
missing_percent = df_after[available_check_cols].isnull().mean() * 100

missing_df_after = pd.DataFrame({
    'Anzahl fehlender Werte': missing_count,
    'Prozent fehlender Werte': missing_percent
})

# Sortieren und nur Zeilen mit fehlenden Werten anzeigen
missing_df_after = missing_df_after[missing_df_after['Anzahl fehlender Werte'] > 0]
missing_df_after = missing_df_after.sort_values(by='Anzahl fehlender Werte', ascending=False)

print("Fehlende Werte NACH IM/GEOCODING (in den bearbeiteten Zeilen):")
print(missing_df_after)

## 8. Weitere Untersuchung der Missing Values

### Wir suchen die 631 Zeilen, welche immer noch leer sind. (NaN)

In [ ]:
# 1. Filtern: Suche nach Zeilen, bei denen CITY ODER STATE fehlt (nach der Imputation in Zelle 6).
# Basierend auf Ihrem Output: 631 Zeilen fehlen CITY und 631 Zeilen fehlen STATE.
missing_city_state = df_imputed['CITY'].isna() | df_imputed['STATE'].isna()

# 2. Filtern der relevanten Geo-Spalten (Basis für die Zuweisung)
check_cols = ['SR_NUMBER', 'STREET_ADDRESS', 'LATITUDE', 'WARD', 'CITY', 'STATE']
df_sample = df_imputed.loc[missing_city_state, check_cols]

# 3. Ausgabe einer Stichprobe (z.B. die ersten 10 Zeilen)
print(f"\n--- Stichprobe der {len(df_sample)} Zeilen, denen CITY/STATE fehlt ---")
print("Beachten Sie: STREET_ADDRESS, LATITUDE und WARD sollten alle fehlen (NaN), da die Imputationslogik diese Basisdaten benötigt.")
# to_string(index=False) anstelle von to_markdown()
print(df_sample.head(10).to_string(index=False))


--- Stichprobe der 631 Zeilen, denen CITY/STATE fehlt ---
Beachten Sie: STREET_ADDRESS, LATITUDE und WARD sollten alle fehlen (NaN), da die Imputationslogik diese Basisdaten benötigt.
    SR_NUMBER STREET_ADDRESS  LATITUDE  WARD CITY STATE
SR24-02337298            NaN       NaN   NaN  NaN   NaN
SR24-02330082            NaN       NaN   NaN  NaN   NaN
SR24-02327658            NaN       NaN   NaN  NaN   NaN
SR24-02326382            NaN       NaN   NaN  NaN   NaN
SR24-02315139            NaN       NaN   NaN  NaN   NaN
SR24-02312297            NaN       NaN   NaN  NaN   NaN
SR24-02312149            NaN       NaN   NaN  NaN   NaN
SR24-02307862            NaN       NaN   NaN  NaN   NaN
SR24-02289798            NaN       NaN   NaN  NaN   NaN
SR24-02285656            NaN       NaN   NaN  NaN   NaN


## 9. Bereinigung der neuen CSV Dateien

### Löschen aller Zeilen die noch NaN haben, die wir nicht imputieren konnten.

In [ ]:
# Liste der CSV-Dateien, die überschrieben werden sollen.
csv_files_to_clean = [ "dataset/imputed_geo_data_only.csv", "dataset/all_data_with_imputed_geo.csv", "dataset/only_211_new_coordinates.csv" ]

# Vollständige Liste der Geo-Spalten, die auf NaN geprüft werden sollen.
geo_cols_to_check_all = [
    'LATITUDE', 
    'LONGITUDE', 
    'X_COORDINATE', 
    'Y_COORDINATE', 
    'LOCATION', 
    'CITY',
    'STATE',
]

for file_path in csv_files_to_clean:
    
    try:
        # 1. Datei laden
        df_csv = pd.read_csv(file_path)
        initial_rows = len(df_csv)

        # Filtere die Spalten, die tatsächlich in der aktuellen CSV-Datei existieren
        available_geo_cols = [c for c in geo_cols_to_check_all if c in df_csv.columns]
        
        # 2. Lösche alle Zeilen, in denen in EINER der Geo-Spalten (available_geo_cols) ein NaN-Wert steht.
        # 'how='any'' sorgt dafür, dass die Zeile bei der ersten gefundenen Lücke gelöscht wird.
        df_cleaned_csv = df_csv.dropna(subset=available_geo_cols, how='any')
        
        final_rows = len(df_cleaned_csv)
        rows_dropped = initial_rows - final_rows
        
        # 3. Bereinigte Daten UNTER DEM GLEICHEN NAMEN speichern (Überschreiben)
        df_cleaned_csv.to_csv(file_path, index=False, encoding="utf-8-sig")
        
        # 4. Ausgabe des Ergebnisses
        print(f"\n Erfolgreich BEREINIGT und ÜBERSCHRIEBEN: '{file_path}'")
        print(f"   -> Überprüfte Spalten: {available_geo_cols}")
        print(f"   -> {rows_dropped} Zeilen gelöscht (fehlende Daten in einer der Geo-Spalten).")
        print(f"   -> Die Datei enthält jetzt {final_rows} Zeilen.")
        
    except FileNotFoundError:
        print(f"\nAchtung: Datei nicht gefunden: {file_path}. Überspringe.")
    except KeyError:
        print(f"\nAchtung: Fehler beim Zugriff auf Spalten in {file_path}. Überspringe.")

print("Bereinigung abgeschlossen. Die Originaldateien wurden überschrieben.")


 Erfolgreich BEREINIGT und ÜBERSCHRIEBEN: 'dataset/imputed_geo_data_only.csv'
   -> Überprüfte Spalten: ['LATITUDE', 'LONGITUDE', 'X_COORDINATE', 'Y_COORDINATE', 'LOCATION', 'CITY', 'STATE']
   -> 100 Zeilen gelöscht (fehlende Daten in einer der Geo-Spalten).
   -> Die Datei enthält jetzt 152344 Zeilen.

 Erfolgreich BEREINIGT und ÜBERSCHRIEBEN: 'dataset/all_data_with_imputed_geo.csv'
   -> Überprüfte Spalten: ['LATITUDE', 'LONGITUDE', 'X_COORDINATE', 'Y_COORDINATE', 'LOCATION', 'CITY', 'STATE']
   -> 804 Zeilen gelöscht (fehlende Daten in einer der Geo-Spalten).
   -> Die Datei enthält jetzt 1913125 Zeilen.

 Erfolgreich BEREINIGT und ÜBERSCHRIEBEN: 'dataset/only_211_new_coordinates.csv'
   -> Überprüfte Spalten: ['LATITUDE', 'LONGITUDE', 'X_COORDINATE', 'Y_COORDINATE', 'LOCATION', 'CITY', 'STATE']
   -> 0 Zeilen gelöscht (fehlende Daten in einer der Geo-Spalten).
   -> Die Datei enthält jetzt 211 Zeilen.

--- Bereinigung abgeschlossen. Die Originaldateien wurden überschrieben. ---


## 10. Überprüfen der restlichen Missing Values

In [21]:

df_after = pd.read_csv("dataset/all_data_with_imputed_geo.csv") 

check_cols = [
    'STREET_ADDRESS', 'LATITUDE', 'LONGITUDE', 
    'X_COORDINATE', 'Y_COORDINATE', 'LOCATION', 'CITY', 'STATE'
]

# Filtern der Spalten, die wirklich im DataFrame sind, um Fehler zu vermeiden
available_check_cols = [c for c in check_cols if c in df_after.columns]

missing_count = df_after[available_check_cols].isnull().sum()
missing_percent = df_after[available_check_cols].isnull().mean() * 100

missing_df_after = pd.DataFrame({
    'Anzahl fehlender Werte': missing_count,
    'Prozent fehlender Werte': missing_percent
})

# Sortieren und nur Zeilen mit fehlenden Werten anzeigen
missing_df_after = missing_df_after[missing_df_after['Anzahl fehlender Werte'] > 0]
missing_df_after = missing_df_after.sort_values(by='Anzahl fehlender Werte', ascending=False)

print("Fehlende Werte NACH IM/GEOCODING (in den bearbeiteten Zeilen):")
print(missing_df_after)

Fehlende Werte NACH IM/GEOCODING (in den bearbeiteten Zeilen):
                Anzahl fehlender Werte  Prozent fehlender Werte
STREET_ADDRESS                      12                 0.000627


In [3]:

CSV_FILE = "dataset/all_data_with_imputed_geo.csv"
try:
    # 1. CSV-Datei laden
    df_csv = pd.read_csv(CSV_FILE)
    
    # 2. Filtern nach Zeilen, bei denen die STREET_ADDRESS fehlt (NaN ist)
    missing_address_mask = df_csv['STREET_ADDRESS'].isna()
    
    # 3. Auswahl relevanter Spalten zur Überprüfung
    check_cols = [
        'SR_NUMBER', 'STREET_ADDRESS', 'STREET_NUMBER', 
        'STREET_NAME', 'STREET_DIRECTION', 'STREET_TYPE', 
        'LATITUDE', 'LONGITUDE', 'CITY'
    ]
    
    # Filtere, um nur die Spalten zu verwenden, die wirklich in der CSV existieren
    available_check_cols = [c for c in check_cols if c in df_csv.columns]
    
    df_missing_address = df_csv.loc[missing_address_mask, available_check_cols]
    
    # 4. Ausgabe des Ergebnisses
    anzahl_fehlend = len(df_missing_address)
    print(f"\n--- {anzahl_fehlend} Zeilen mit fehlender STREET_ADDRESS (NaN) ---")
    
    if anzahl_fehlend > 0:
        # Zeige die ersten 50 Zeilen (oder alle 12 Zeilen, wenn die Anzahl stimmt)
        print(df_missing_address.head(50).to_string(index=False))
    else:
        print("Keine Zeilen mit fehlender STREET_ADDRESS in dieser CSV gefunden.")

except FileNotFoundError:
    print(f"\nFEHLER: Die Datei {CSV_FILE} wurde nicht gefunden. Stellen Sie sicher, dass der Pfad korrekt ist und die Datei existiert.")


--- 12 Zeilen mit fehlender STREET_ADDRESS (NaN) ---
    SR_NUMBER STREET_ADDRESS STREET_NUMBER STREET_NAME STREET_DIRECTION STREET_TYPE  LATITUDE  LONGITUDE    CITY
SR24-02333234            NaN           NaN         NaN              NaN         NaN 41.994271 -87.679933 Chicago
SR24-01986028            NaN           NaN         NaN              NaN         NaN 41.827544 -87.611632 Chicago
SR24-01681103            NaN           NaN         NaN              NaN         NaN 41.994271 -87.679933 Chicago
SR24-01389933            NaN           NaN         NaN              NaN         NaN 41.913638 -87.710186 Chicago
SR24-01378822            NaN           NaN         NaN              NaN         NaN 41.994271 -87.679933 Chicago
SR24-01377294            NaN           NaN         NaN              NaN         NaN 41.913638 -87.710186 Chicago
SR24-01205862            NaN           NaN         NaN              NaN         NaN 41.895062 -87.650132 Chicago
SR24-01170340            NaN           NaN

## 11. Mithilfe von Geocoding Adressen befüllen

In [ ]:
CSV_FILE_TO_UPDATE = "dataset/all_data_with_imputed_geo.csv"
geolocator = Nominatim(user_agent="chicago_311_csv_update") 

def reverse_geocode_csv(file_path):
    
    try:
        # 1. Datei laden
        df_update = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"FEHLER: Datei {file_path} nicht gefunden. Abbruch.")
        return

    # 2. Filtern: Suche nach Zeilen, die LAT/LON haben, aber keine STREET_ADDRESS
    to_reverse_geocode = (df_update['STREET_ADDRESS'].isna()) & \
                         (df_update['LATITUDE'].notna()) & \
                         (df_update['LONGITUDE'].notna())
    
    count_to_process = to_reverse_geocode.sum()
    if count_to_process == 0:
        print("Keine Zeilen für Reverse Geocoding in dieser CSV gefunden. Keine Änderung vorgenommen.")
        return
        
    print(f"Verarbeite {count_to_process} Adressen via Reverse Geocoding...")
    
    imputed_count = 0

    # 3. Iteration nur über die betroffenen Zeilen im temporären DataFrame
    for idx in df_update[to_reverse_geocode].index:
        lat = df_update.loc[idx, 'LATITUDE']
        lon = df_update.loc[idx, 'LONGITUDE']
        
        try:
            location = geolocator.reverse((lat, lon), timeout=15) 
            
            if location and location.address:
                df_update.loc[idx, 'STREET_ADDRESS'] = location.address
                
                # Versuche, Komponenten zu extrahieren (falls vorhanden)
                address_details = location.raw.get('address', {})
                df_update.loc[idx, 'STREET_NUMBER'] = address_details.get('house_number', pd.NA)
                df_update.loc[idx, 'STREET_NAME'] = address_details.get('road', pd.NA)
                
                imputed_count += 1
                
        except Exception:
            # Bei Fehlern die Zeile überspringen
            pass

        # Warte 1 Sekunde zur Vermeidung von Rate Limiting
        time.sleep(1) 
        
    print(f"Reverse Geocoding abgeschlossen. {imputed_count} Adressen erfolgreich imputiert.")

    # 4. CSV-Datei überschreiben
    df_update.to_csv(file_path, index=False, encoding="utf-8-sig")

    print(f"Verbleibende fehlende Adressen nach diesem Schritt: {df_update['STREET_ADDRESS'].isna().sum()}")

reverse_geocode_csv(CSV_FILE_TO_UPDATE)

Verarbeite 12 Adressen via Reverse Geocoding...
Reverse Geocoding abgeschlossen. 12 Adressen erfolgreich imputiert.
Verbleibende fehlende Adressen nach diesem Schritt: 0


In [6]:
import pandas as pd

files = ["dataset/imputed_geo_data_only.csv", "dataset/all_data_with_imputed_geo.csv", "dataset/only_211_new_coordinates.csv"]
geo_cols = ['LATITUDE', 'LONGITUDE', 'X_COORDINATE', 'Y_COORDINATE', 'LOCATION', 'STATE', 'CITY']

for file in files:
    df = pd.read_csv(file)
    available_cols = [c for c in geo_cols if c in df.columns]
    total_nan = df.loc[:, available_cols].isna().sum().sum()
    print(f"'{file}': Gesamt fehlende Geo-Werte (NaN): {total_nan}")

'dataset/imputed_geo_data_only.csv': Gesamt fehlende Geo-Werte (NaN): 0
'dataset/all_data_with_imputed_geo.csv': Gesamt fehlende Geo-Werte (NaN): 0
'dataset/only_211_new_coordinates.csv': Gesamt fehlende Geo-Werte (NaN): 0


Jetzt haben wir die drei CSV Files komplett imputiert mithilfe von

- Neovatim: Street Address Latitude und Longitude hinzugefügt und Reverse

- Transformer: wenn Teile vergessen wurden, aber schon in anderen (mathematisch gleichen) Spalten standen wurden sie in die anderen rübergefügt( Beispiel X und Y Koordinaten und Latitude und Longitude z B so: transformiert von EPSG:4326 (LAT/LONG) zu EPSG:3435 (X/Y))

- und zum Schluss löschen der NaN